# メールを送信する(SMTP)

In [1]:
import smtplib

In [13]:
smtp_obj = smtplib.SMTP("smtp-mail.outlook.com", 587)

In [14]:
smtp_obj.ehlo()

(250,
 b'OSAPR01CA0297.outlook.office365.com Hello [202.220.176.101]\nSIZE 157286400\nPIPELINING\nDSN\nENHANCEDSTATUSCODES\nSTARTTLS\n8BITMIME\nBINARYMIME\nCHUNKING\nSMTPUTF8')

In [15]:
smtp_obj.starttls()

(220, b'2.0.0 SMTP server ready')

In [16]:
smtp_obj.login("kumakiyo000@outlook.com", "kiyotaka000")

(235, b'2.7.0 Authentication successful')

In [17]:
smtp_obj.sendmail("kumakiyo000@outlook.com", "kumakiyo000@gmail.com", 
                  "Subject: So long.\nDear Alice, so long and thanks for all the fish. Sincerely, Bob")

{}

In [18]:
smtp_obj.quit()

(221, b'2.0.0 Service closing transmission channel')

## SMTPサーバーに接続する

In [33]:
import smtplib
smtp_obj = smtplib.SMTP("smtp-mail.outlook.com", 587)
type(smtp_obj)

smtplib.SMTP

メールを送信するにはまずSMTPサーバーに接続する必要がある

そのためには smtplib モジュールの SMTP 関数に対応するSMTPサーバー名とポート番号を渡すことでできる、SMTPオブジェクトを生成する必要がある

SMTPサーバーとポート番号はメールを送るのに必要な情報で、基本、SMTPサーバーはメールプロバイダーのドメイン名にsmtpがついたもので、ポート番号はコマンドを暗号化するための標準規格TLSを使う587番になっている

TLSをサポートしていない場合は、`smtplib.SMTP_SSL`を使ってポート番号を465を指定して生成する

接続してから数分間経つと自動的に切断されるので、また接続しないといけなくなるので注意

主なメールプロバイダーとSMTPサーバー一覧

|プロバイダー|SMTPサーバー|
|:-|:-|
|Gmail|smtp.gmail.com|
|Outlook.com\/Hotmail.com|smtp-mail.outlook.com|
|Yahoo Mail|smtp.mail.yahoo.com|
|AT&T|amtp.mail.att.net(ポート番号465)|
|Comcast|smtp.comcast.net|
|Verizon|smtp.verizon.net(ポート番号465)|

## SMTPに挨拶を送る

In [34]:
smtp_obj.ehlo()

(250,
 b'OSAPR01CA0271.outlook.office365.com Hello [202.220.176.101]\nSIZE 157286400\nPIPELINING\nDSN\nENHANCEDSTATUSCODES\nSTARTTLS\n8BITMIME\nBINARYMIME\nCHUNKING\nSMTPUTF8')

SMTPオブジェクト生成したら ehlo メソッドを呼んで、SMTPにhelloと挨拶のようなものを送らなければならない

250が返ってくれば成功したことになる

## TLS暗号化の開始

In [35]:
smtp_obj.starttls()

(220, b'2.0.0 SMTP server ready')

ポート番号を587番で接続した場合、TLS暗号化を使用することになるので暗号化を有効にするために、  
SMTPオブジェクトの starttls メソッドを呼ぶ必要がある(SSLを使う場合は設定不要)

220が返っていくれば成功したことになる

## SMTPサーバーにログインする

In [36]:
smtp_obj.login("kumakiyo000@outlook.com", "kiyotaka000")

(235, b'2.7.0 Authentication successful')

SMTPサーバーと暗号化接続ができたら、ユーザー名とパスワードをSMTPオブジェクトの login メソッドに渡してログインする

235が返ってくれば認証が成功したことになる

## メールを送信する

In [37]:
smtp_obj.sendmail("kumakiyo000@outlook.com", "kumakiyo000@gmail.com", 
                  "Subject: So long.\nDear Alice, so long and thanks for all the fish. Sincerely, Bob")

{}

メールを送信するにはSMTPオブジェクトの sendmail メソッドをよび、3つの引数を渡す必要がある

渡す引数は、送信者のメールアドレス、受信者のメールアドレス(複数人いる場合はリストで渡す)、メールの本文を文字列で渡す

メールの本文は必ず Subject: ～\n から始め、改行前のメールの件名と改行後の本文を分けなければならない

メールの送信が成功すれば空の辞書が返ってくる

失敗した場合は、宛先ごとのペアが返ってくる

上の方法では日本語で送れないので、送りたい場合は email パッケージの mine.text.MINEText や header.Header などを使う必要がある

## SMTPサーバーから切断する

In [38]:
smtp_obj.quit()

(221, b'2.0.0 Service closing transmission channel')

メールを送信したら quit メソッドを呼んで終了する

221が返ってくればセッションが終了したことを意味する

# メールを取得・削除する(IMAP)

In [3]:
import imapclient, ssl

In [17]:
context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
imap_obj = imapclient.IMAPClient("imap-mail.outlook.com", ssl=True, ssl_context=context)
imap_obj.login("kumakiyo000@outlook.com","kiyotaka000")

b'LOGIN completed.'

In [18]:
imap_obj.select_folder("INBOX", readonly=True)

{b'PERMANENTFLAGS': (),
 b'EXISTS': 2,
 b'RECENT': 2,
 b'FLAGS': (b'\\Seen',
  b'\\Answered',
  b'\\Flagged',
  b'\\Deleted',
  b'\\Draft',
  b'$MDNSent'),
 b'UNSEEN': [b'1'],
 b'UIDVALIDITY': 14,
 b'UIDNEXT': 33,
 b'READ-ONLY': [b'']}

In [19]:
UIDs = imap_obj.search(["SINCE", "04-Jun-2020"])
UIDs

[27, 29]

In [34]:
raw_messages = imap_obj.fetch([27], ["BODY[]", "FLAGS"])

In [21]:
import pyzmail

In [35]:
message = pyzmail.PyzMessage.factory(raw_messages[27][b"BODY[]"])
message.get_subject()

'もう少しで新しい OneDrive のセットアップ完了です'

In [36]:
message.get_addresses("from")

[('Microsoft OneDrive', 'email@mail.onedrive.com')]

In [37]:
message.get_addresses("to")

[('kumakiyo127@outlook.com', 'kumakiyo127@outlook.com')]

In [38]:
message.get_addresses("cc")

[]

In [39]:
message.get_addresses("bcc")

[]

In [40]:
message.text_part != None

False

In [41]:
message.text_part.get_payload().decode(message.text_part.charset)

AttributeError: 'NoneType' object has no attribute 'get_payload'

In [42]:
message.html_part != None

True

In [43]:
message.html_part.get_payload().decode(message.html_part.charset)

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml" dir="ltr" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office"><head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r\n<meta name="format-detection" content="telephone=no">\r\n<meta name="format-detection" content="date=no">\r\n<meta name="format-detection" content="address=no">\r\n<meta name="format-detection" content="email=no">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\r\n<meta name="robots" content="noindex, nofollow">\r\n<meta name="x-apple-disable-message-reformatting">\r\n\r\n\r\n<title></title>\r\n<!--[if gte mso 9]><xml>\r\n  <o:OfficeDocumentSettings>\r\n    <o:AllowPNG/>\r\n    <o:PixelsPerInch>96</o:PixelsPerInch>\r\n  </o:OfficeDocumentSettings>\r\n</xml><![endif

In [44]:
imap_obj.logout()

b'Microsoft Exchange Server IMAP4 server signing off.'

## IMAPサーバーに接続する

In [37]:
import imapclient, ssl
context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
imap_obj = imapclient.IMAPClient("imap-mail.outlook.com", ssl=True, ssl_context=context)

IMAPサーバに接続するには imapclient モジュールの IMAPClient 関数を呼び IMAPClient オブジェクトを生成する必要がある

IMAPClient関数に渡すのはIMAPサーバー名で、ほとんどのメールプロバイダーはSSL暗号化を要求するのでキーワード引数のsslにTrueを渡している

主なメールプロバイダーとIMAPサーバー一覧

|プロバイダー|IMAPサーバー|
|:-|:-|
|Gmail|imap.gmail.com|
|Outlook.com\/Hotmail.com|imap-mail.outlook.com|
|Yahoo Mail|imap.mail.yahoo.com|
|AT&T|imap.mail.att.net|
|Comcast|imap.comcast.net|
|Verizon|imap.verizon.net|

## IMAPサーバーにログインする

In [38]:
imap_obj.login("kumakiyo000@outlook.com","kiyotaka000")

b'LOGIN completed.'

IMAPClientオブジェクトの login メソッドにユーザー名とパスワードを渡してログインする

## メールを探す

### フォルダを選択する

In [4]:
import pprint
pprint.pprint(imap_obj.list_folders())

[((b'\\HasNoChildren',), b'/', 'アーカイブ'),
 ((b'\\HasNoChildren',), b'/', 'Notes'),
 ((b'\\HasNoChildren', b'\\Drafts'), b'/', 'Drafts'),
 ((b'\\HasNoChildren', b'\\Trash'), b'/', 'Deleted'),
 ((b'\\Marked', b'\\HasNoChildren'), b'/', 'Inbox'),
 ((b'\\HasNoChildren',), b'/', 'Outbox'),
 ((b'\\HasNoChildren', b'\\Sent'), b'/', 'Sent'),
 ((b'\\HasNoChildren', b'\\Junk'), b'/', 'Junk')]


IMAPClientオブジェクトの list_folders メソッドを使うと所持しているフォルダのリストを3つの値をタプルにまとめて返してくれる

1つ目の値がフォルダのフラグを表すタプル、2つ目の値が親フォルダとサブフォルダを分けるのに使われている区切り文字、3つ目がフォルダの全体名になる

In [5]:
imap_obj.select_folder("Inbox", readonly=True)

{b'PERMANENTFLAGS': (),
 b'EXISTS': 2,
 b'RECENT': 2,
 b'FLAGS': (b'\\Seen',
  b'\\Answered',
  b'\\Flagged',
  b'\\Deleted',
  b'\\Draft',
  b'$MDNSent'),
 b'UNSEEN': [b'1'],
 b'UIDVALIDITY': 14,
 b'UIDNEXT': 33,
 b'READ-ONLY': [b'']}

検索対象の選択するにはIMAPClientオブジェクトの select_folder メソッドに上のフォルダ名を渡すことでできる

キーワード引数のreadonlyにTrueを渡すことで、以後のメソッド呼ぶだしでメールの変更や削除をしてしまうことを防ぐことができる

### 検索を実行する

In [55]:
imap_obj.search(["ALL"]) # フォルダ内のすべてのメッセージを返す

[27, 29]

In [57]:
imap_obj.search(["ON", "04-JUN-2020"]) # 2020年6月4日のすべてのメッセージを返す

[27, 29]

In [56]:
imap_obj.search(["SINCE", "01-Jun-2020", "FROM", "01-JUl", "UNSEEN"]) # 2020年6月1日から2020年7月1日の未読メッセージを返す

[]

In [58]:
imap_obj.search(["SINCE", "01-JAN-2020", "FROM", "kumakiyo000@outlook.com"]) # 2020年1月1日以降のkumakiyo000からきたメッセージを返す

[29]

In [59]:
imap_obj.search(["SINCE", "01-JAN-2020", "NOT", "FROM", "kumakiyo000@outlook.com"]) 
# 2020年1月1日以降のkumakiyo000以外からきたメッセージを返す

[27]

In [60]:
imap_obj.search(["OR", "FROM", "kumakiyo000@outlook.com", "FROM", "kumakiyo000@gmail.com"]) 
# 1つ目か2つ目のどちらかから来たメッセージを返す

[29]

IMAPClientオブジェクトの search メソッドに検索キーをリストのして渡すことで、マッチした特定のメッセージを取得することができる

複数の検索キーを渡すこともでき、その場合はすべての検索結果にマッチしたメッセージを返す

IMAP検索キー一覧

|検索キー|説明|
|:-|:-|
|"ALL"|フォルダ内の全メールを返す|
|"BEFORE","*date*"|指定した日付(date)よりも前のメッセージを返す(当日は含まれない)|
|"ON","*date*"|指定した日付(date)当日のメッセージを返す|
|"SINCE","*date*"|指定した日付(date)以降のメッセージを返す(当日も含む)|
|"SUBJECT","*string*"|文字列(string)が件名に含まれているメッセージを返す|
|"BODY","*string*"|文字列(string)が本文に含まれているメッセージを返す|
|"TEXT","*string*"|文字列(string)が件名か本文のどちらかに含まれているメッセージを返す|
|"FROM","*string*"|文字列(string)が差出人のメールアドレスに含まれているメッセージを返す|
|"TO","*string*"|文字列(string)が宛先のメールアドレスに含まれているメッセージを返す|
|"CC","*string*"|文字列(string)が同報のメールアドレスに含まれているメッセージを返す|
|"BCC","*string*"|文字列(string)が秘匿同報のメールアドレスに含まれているメッセージを返す|
|"SEEN"|既読メッセージを返す|
|"UNSEEN"|未読メッセージを返す|
|"ANSWERED"|返信済みメッセージを返す|
|"UNANSWERED"|未返信メッセージを返す|
|"DELETED"|削除済みメッセージを返す|
|"UNDELETED"|未削除のメッセージを返す|
|"DRAFT"|下書きメッセージを返す|
|"UNDRAFT"|下書きメッセージではないものを返す|
|"FLAGGED"|フラグのついたメッセージを返す|
|"UNFLAGGED"|フラグのついていないメッセージを返す|
|"LARGER","*N*"|Nバイトよりも大きいメッセージを返す|
|"SMALLER","*N*"|Nバイトよりも小さいメッセージを返す|
|"NOT","*search-key*"|検索キー(search-key)にマッチしないメッセージを返す|
|"OR","*search-key1,search-key2*"|どちらかの検索キーにマッチするメッセージを返す|

In [6]:
UIDs = imap_obj.search(["SINCE", "01-JAN-2020"])
UIDs

[27, 29]

searchメソッドはメールそのものを返すのではなく、整数値のユニークなID(UID)を返す

UIDを ferch メソッドに渡すことでメールの内容を取得できる

### サイズの制限

In [1]:
import imaplib
imaplib._MAXLINE = 10000000

検索結果が1万バイトを超えると、サイズ制限に引っかかってエラーを起こしてしまう

この制限はメモリーを使い切らないようにするために設けられているが、小さすぎて困ることもある

そのときは imaplib モジュールの \_MAXLINE アトリビュートの値を変えることで制限を変えることができる(上では10万に変更)

## メールを取得して既読マークを付ける

In [7]:
raw_messages = imap_obj.fetch(UIDs, ["BODY[]"])

In [8]:
import pprint
pprint.pprint(raw_messages)

defaultdict(<class 'dict'>,
            {27: {b'BODY[]': b'Received: from MW2NAM10HT221.eop-nam10.prod.prot'
                             b'ection.outlook.com\r\n (2603:1096:4:be::28) by'
                             b' SG2PR03MB5069.apcprd03.prod.outlook.com with HT'
                             b'TPS\r\n via SG3P274CA0016.SGPP274.PROD.OUTLOOK'
                             b'.COM; Thu, 4 Jun 2020 13:35:51 +0000\r\nReceiv'
                             b'ed: from MW2NAM10FT037.eop-nam10.prod.protection'
                             b'.outlook.com\r\n (2a01:111:e400:7e87::44) by\r\n'
                             b' MW2NAM10HT221.eop-nam10.prod.protection.outlook'
                             b'.com (2a01:111:e400:7e87::495)\r\n with Micros'
                             b'oft SMTP Server (version=TLS1_2,\r\n cipher=TL'
                             b'S_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.30'
                             b'21.23; Thu, 4 Jun\r\n 2020 13:35:50 +0000\r\nAut'
          

UIDのリストをIMAPClientオブジェクトの fetch メソッドの第一引数に渡して、  
第二引数に本文を取り出すための["BODY[]"]を渡すことで内容を取得することができる

取り出してもバイト型で分かりにくい内容になっているので、後でpyzmailモジュールを使って見やすくする

In [10]:
imap_obj.select_folder("Inbox", readonly=False)

{b'PERMANENTFLAGS': (b'\\Seen',
  b'\\Answered',
  b'\\Flagged',
  b'\\Deleted',
  b'\\Draft',
  b'$MDNSent'),
 b'EXISTS': 2,
 b'RECENT': 2,
 b'FLAGS': (b'\\Seen',
  b'\\Answered',
  b'\\Flagged',
  b'\\Deleted',
  b'\\Draft',
  b'$MDNSent'),
 b'UNSEEN': [b'1'],
 b'UIDVALIDITY': 14,
 b'UIDNEXT': 33,
 b'READ-WRITE': True}

前にselect_folderに対してreadonlyをTrueにしたので、未読状態も変更されない

そのため、もう一度select_folderを呼んでreadonlyに対してFalseを入れることで既読がつけられる

## 生のメールからメールアドレスを取得する

In [12]:
import pyzmail
message = pyzmail.PyzMessage.factory(raw_messages[27][b"BODY[]"])

In [13]:
message.get_subject()

'もう少しで新しい OneDrive のセットアップ完了です'

In [14]:
message.get_address("from")

('Microsoft OneDrive', 'email@mail.onedrive.com')

In [15]:
message.get_address("to")

('kumakiyo127@outlook.com', 'kumakiyo127@outlook.com')

In [16]:
message.get_address("cc")

('', '')

In [17]:
message.get_address("bcc")

('', '')

pyzmailモジュールの PyzMessage.factory 関数にバイト型のメッセージのUID番号と内容の入っているb"BODY[]"を指定して渡すと PyzMessage オブジェクトを生成できる

PyzMessageオブジェクトには件名を返す get_subject メソッドや、送信者や受信者を返す get_address メソッドなどがある

## 生のメッセージから本文を取り出す

In [18]:
message1 = pyzmail.PyzMessage.factory(raw_messages[27][b"BODY[]"])
message2 = pyzmail.PyzMessage.factory(raw_messages[29][b"BODY[]"])

In [24]:
message1.text_part != None

False

In [32]:
message2.text_part != None

True

In [26]:
message1.html_part != None

True

In [27]:
message2.html_part != None

False

メールはプレーンテキストやHTML、または両方の形式で送ることができる

PyzMessageオブジェクトの text_part アトリビュートは、テキストが含まれていない場合にNoneを返す(つまりHTMLのみ)  
逆に html_part アトリビュートは、HTMLが含まれていない場合にNoneを返す(つまりテキストのみ)

つまりmessage1はHTMLのみで、message2はテキストのみということがわかる

In [31]:
message2.text_part.get_payload().decode(message2.text_part.charset) # 空メールなので表示されなかった

TypeError: decode() argument 1 must be str, not None

In [33]:
message1.html_part.get_payload().decode(message1.html_part.charset)

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml" dir="ltr" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office"><head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r\n<meta name="format-detection" content="telephone=no">\r\n<meta name="format-detection" content="date=no">\r\n<meta name="format-detection" content="address=no">\r\n<meta name="format-detection" content="email=no">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\r\n<meta name="robots" content="noindex, nofollow">\r\n<meta name="x-apple-disable-message-reformatting">\r\n\r\n\r\n<title></title>\r\n<!--[if gte mso 9]><xml>\r\n  <o:OfficeDocumentSettings>\r\n    <o:AllowPNG/>\r\n    <o:PixelsPerInch>96</o:PixelsPerInch>\r\n  </o:OfficeDocumentSettings>\r\n</xml><![endif

テキストを呼び出すなら、PyzMessageオブジェクトのtext_partアトリビュートの get_payload メソッドを呼ぶ  
その戻り値はバイト型なので decode メソッドを呼び、PyzMessageオブジェクトのテキストのエンコードが入っている charset アトリビュートを渡している(上では"utf-8"が渡されている)

同じようにHTMLを呼び出すならhtml_partからget_payloadメソッドを呼び出すことでできる

## メールを削除する

In [40]:
imap_obj.select_folder("Inbox", readonly=False)
UIDs = imap_obj.search(["ON", "06-JUN-2020"])
UIDs

[33]

In [42]:
imap_obj.delete_messages(UIDs)

{}

In [43]:
imap_obj.expunge()

(b'EXPUNGE completed.', [(3, b'EXISTS'), (3, b'EXPUNGE'), (2, b'EXISTS')])

メールを削除するには変更を許可する必要があるのでIMAPClientオブジェクトのselect_folderでフォルダを指定するときにreadonlyをFalseにする必要がある

そしてメールをsearchメソッドで検索しメッセージIDを取得する

IMAPClientオブジェクトの delete_messages メソッドにそのメッセージIDを渡すことでそのメールをゴミ箱に移し、  
IMAPClientオブジェクトの expunge メソッドを呼ぶことで、ごみ箱の中身を完全に削除することができる

## IMAPサーバーから接続を切断する

In [44]:
imap_obj.logout()

b'Microsoft Exchange Server IMAP4 server signing off.'

IMAPサーバーを切断するにはIMAPClientオブジェクトの logout メソッドを呼ぶことでできる

# Twilioを使ってSMSを送る

Twilioを利用しないので以下は想定の結果のみ

## SMSを送信する

In [1]:
from twilio.rest import Client

In [ ]:
account_SID = "アカウントSIDを入れる"
auth_token = "認証トークンを入れる"
twilio_cli = Client(account_SID, auth_token)
my_twilio_number = "+12345678901" # 購入した米国の電話番号
my_cell_phone = "+819012345678" # 自分の携帯電話番号
message = twilio_cli.messages.create(body="メッセージをここに入れる", from_=my_twilio_number, to=mycellphone)

SMSを送るには、twilio.rest から Client をインポートする必要がある

そして、Client関数に カウントSIDと認証トークンを渡してClientオブジェクトを生成する

Clientオブジェクトにキーワード引数のbodyにメッセージ本文、from_に自分のtwilioの電話番号、toに送る相手の電話番号を渡すことで送ることができる